In [1]:
### Author: Pongpisit Thanasutives ###
from itertools import combinations
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from tqdm import trange

In [2]:
def TopRsq(X_full, y, m, n_tops=25):
    n_feats = X_full.shape[-1]
    r_scores = []
    models = []
    for comb in combinations(range(n_feats), m):
        comb = list(comb)
        active_indices = np.zeros(n_feats)
        active_indices[comb] = 1
        X_sub = X_full[:, comb]
        lr = LinearRegression(fit_intercept=False).fit(X_sub, y)
        R2 = lr.score(X_sub, y)
        r_scores.append(R2)
        models.append(active_indices)
    r_scores = np.array(r_scores)
    r_argsort = np.argsort(r_scores)[::-1][:n_tops]
    r_scores = r_scores[r_argsort]
    models = np.array(models).T
    models = models[:, r_argsort]
    rating = np.dot(models, r_scores)
    return models, r_scores, rating

In [3]:
def comprehensive_search(X_full, y, max_support_size=8, n_tops=None, threshold=0.75):
    X = X_full.copy()
    n_feats = X_full.shape[-1]
    n_tops = int(np.ceil(n_feats/2)) if n_tops is None else n_tops
    ratings = np.zeros((n_feats, max_support_size))
    search = True; support_size = 1
    optimal_indices = None
    active_indices = [_ for _ in range(n_feats)]
    while search and support_size <= max_support_size:
        _, _, rating = TopRsq(X, y, m=support_size, n_tops=n_tops)
        rating = rating/rating.max()
        ratings[:, support_size-1][active_indices] = rating
        if support_size >= 2:
            i0 = np.where(ratings[:, support_size-1] + ratings[:, support_size-2] == 0.)[0]
            active_indices = [_ for _ in active_indices if _ not in set(i0)]
            X = X_full[:, active_indices]
            i1 = np.where(ratings[:, support_size-1] > 0)[0]
            i2 = np.where(ratings[:, support_size-2] > 0)[0]
            if len(i1) == len(i2) and np.all(i1 == i2):
                search = False
                optimal_indices = np.where(ratings[:, support_size-1] > threshold)[0]
                if len(optimal_indices) == 0:
                    optimal_indices = None
                    print("No term whose improtance is greater than the threshold...")
        support_size += 1
    if optimal_indices is None:
        print("Not converged...")
    return optimal_indices

In [4]:
n_experiments = 100
n_features = 8
n_informative = 2

threshold = 0.75

success = 0
for i in trange(n_experiments):
    X_train, y_train = make_regression(n_samples=1000, n_features=n_features, n_informative=n_informative)
    top_models, _, _ = TopRsq(X_train, y_train, m=2)
    true_indices = np.where(top_models[:, 0] > 0)[0]
    est_indices = comprehensive_search(X_train, y_train, max_support_size=8, threshold=threshold)
    if est_indices is not None and len(true_indices) == len(est_indices) and np.all(true_indices == est_indices):
        success += 1
        
success/n_experiments

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.89it/s]


0.96